In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import pickle

# Đọc dữ liệu từ dataset
data = pd.read_csv('health_data.csv')

# Chia dữ liệu thành features và labels
X = data.iloc[:, :-1].values  # Features
y = data['health_status'].values  # Labels

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Chuyển đổi nhãn về dạng số
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển đổi nhãn thành dạng one-hot encoding
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Xây dựng mô hình Neural Network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Biên dịch mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


# Save the model structure and weights
model_file = 'model.pkl'
pickle.dump(model, open(model_file, 'wb'))

# Save the scaler object for preprocessing
scaler_file = 'scaler.pkl'
pickle.dump(scaler, open(scaler_file, 'wb'))

# Save the label encoder object for decoding predictions
encoder_file = 'label_encoder.pkl'
pickle.dump(label_encoder, open(encoder_file, 'wb'))



# Đánh giá mô hình trên tập kiểm tra
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Đánh giá mô hình
accuracy = accuracy_score(y_test_classes, y_pred_classes)
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
class_report = classification_report(y_test_classes, y_pred_classes)

# In kết quả
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')


# Dự đoán trạng thái sức khỏe trên dữ liệu mới
new_data = np.array([[30, 85, 94, 38.0, 11, 92, 39.8, 88, 90]])  # Thay thế bằng dữ liệu mới của bạn
new_data = scaler.transform(new_data)

# Dự đoán
predicted_probabilities = model.predict(new_data)
predicted_class_index = np.argmax(predicted_probabilities)
predicted_class = label_encoder.classes_[predicted_class_index]

# In kết quả
print(f'Predicted Health Status: {predicted_class}')




Epoch 1/100


3/3 [==============================] - 1s 147ms/step - loss: 1.4767 - accuracy: 0.2500 - val_loss: 1.4386 - val_accuracy: 0.2000
Epoch 2/100
3/3 [==============================] - 0s 26ms/step - loss: 1.4239 - accuracy: 0.3125 - val_loss: 1.4202 - val_accuracy: 0.2500
Epoch 3/100
3/3 [==============================] - 0s 22ms/step - loss: 1.3827 - accuracy: 0.3000 - val_loss: 1.4086 - val_accuracy: 0.2000
Epoch 4/100
3/3 [==============================] - 0s 19ms/step - loss: 1.3462 - accuracy: 0.3625 - val_loss: 1.4035 - val_accuracy: 0.2000
Epoch 5/100
3/3 [==============================] - 0s 20ms/step - loss: 1.3131 - accuracy: 0.3625 - val_loss: 1.4014 - val_accuracy: 0.2000
Epoch 6/100
3/3 [==============================] - 0s 27ms/step - loss: 1.2824 - accuracy: 0.3875 - val_loss: 1.4004 - val_accuracy: 0.2000
Epoch 7/100
3/3 [==============================] - 0s 20ms/step - loss: 1.2557 - accuracy: 0.4500 - val_loss: 1.4002 - val_accuracy: 0.2000
Epoch 8/100
3/

3/3 [==============================] - 0s 20ms/step - loss: 0.6115 - accuracy: 0.8500 - val_loss: 1.7884 - val_accuracy: 0.2000
Epoch 52/100
3/3 [==============================] - 0s 21ms/step - loss: 0.6007 - accuracy: 0.8500 - val_loss: 1.8049 - val_accuracy: 0.2000
Epoch 53/100
3/3 [==============================] - 0s 19ms/step - loss: 0.5898 - accuracy: 0.8625 - val_loss: 1.8206 - val_accuracy: 0.2000
Epoch 54/100
3/3 [==============================] - 0s 19ms/step - loss: 0.5782 - accuracy: 0.8625 - val_loss: 1.8357 - val_accuracy: 0.2000
Epoch 55/100
3/3 [==============================] - 0s 18ms/step - loss: 0.5678 - accuracy: 0.8750 - val_loss: 1.8548 - val_accuracy: 0.2000
Epoch 56/100
3/3 [==============================] - 0s 18ms/step - loss: 0.5566 - accuracy: 0.8875 - val_loss: 1.8670 - val_accuracy: 0.2000
Epoch 57/100
3/3 [==============================] - 0s 18ms/step - loss: 0.5467 - accuracy: 0.8750 - val_loss: 1.8811 - val_accuracy: 0.2000
Epoch 58/100
3/3 [========

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import pickle


app = Flask(__name__)

model = pickle.load(open('model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))
label_encoder = pickle.load(open('label_encoder.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def predict():
    new_data = request.get_json()  # Nhận dữ liệu mới từ request
    new_data = np.array(new_data['data'])  # Chuyển đổi sang mảng numpy

    if scaler:
        new_data = scaler.transform(new_data)  # Chuẩn hóa dữ liệu nếu có

    predicted_probabilities = model.predict(new_data)
    predicted_class_index = np.argmax(predicted_probabilities)
    predicted_class = label_encoder.classes_[predicted_class_index]  # Chuyển đổi nhãn nếu có

    return jsonify({'predicted_class': predicted_class})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # Chạy trên server mặc định

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.8.101:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 61ms/step


127.0.0.1 - - [24/Dec/2023 00:15:39] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 127ms/step


...

127.0.0.1 - - [24/Dec/2023 00:49:25] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 93ms/step


127.0.0.1 - - [24/Dec/2023 00:52:18] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 36ms/step


127.0.0.1 - - [24/Dec/2023 00:52:48] "POST /predict HTTP/1.1" 200 -
